In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

In [27]:
# ==========================================
# 1. CONFIGURAÇÕES E DIRETÓRIOS
# ==========================================
# Aponte diretamente para a pasta que contém as 4 classes
DATASET_PATH = './AugmentedAlzheimerDataset' 

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.0001

In [28]:
# Criamos um gerador com validation_split (80% Treino / 20% Validação)
# Nota: Usamos seed fixo para garantir que o treino e teste não se misturem
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,    # <--- Separa 20% para teste automaticamente
    rotation_range=10,       # Data Augmentation no treino
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Gerador separado para validação (sem augmentation, apenas preprocessamento)
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2     # <--- Importante ter o mesmo split aqui
)

print("Carregando conjunto de TREINO (80%)...")
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,            # Pasta raiz
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',       # <--- Define que este é o grupo de TREINO
    seed=42,                 # Seed garante que a divisão seja igual nos dois
    shuffle=True
)

print("Carregando conjunto de VALIDAÇÃO (20%)...")
validation_generator = val_datagen.flow_from_directory(
    DATASET_PATH,            # Mesma pasta raiz
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',     # <--- Define que este é o grupo de VALIDAÇÃO
    seed=42,                 # Mesmo seed do treino
    shuffle=False
)

# Verifica se as classes foram encontradas corretamente
print(f"Classes encontradas: {train_generator.class_indices}")

Carregando conjunto de TREINO (80%)...
Found 27188 images belonging to 4 classes.
Carregando conjunto de VALIDAÇÃO (20%)...
Found 6796 images belonging to 4 classes.
Classes encontradas: {'MildDemented': 0, 'ModerateDemented': 1, 'NonDemented': 2, 'VeryMildDemented': 3}


In [29]:
# 3. MODELO (TRANSFER LEARNING - RESNET50)
# ==========================================
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
# ==========================================
# 4. TREINAMENTO
# ==========================================
print("\nIniciando Treinamento...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator # Agora usamos o validation_generator criado acima
)


Iniciando Treinamento...
Epoch 1/10
 30/850 ━━━━━━━━━━━━━━━━━━━━ 28:21 2s/step - accuracy: 0.2561 - loss: 1.8995

KeyboardInterrupt: 

In [ ]:
# ==========================================
# 5. SALVAR
# ==========================================
model.save('resnet_alzheimer_augmented.h5')
print("Modelo salvo.")